Import Libraries

In [2]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import math 
import cv2
import pytesseract

Get all the property addresses

In [65]:
# rightmove main URL
from decimal import ROUND_UP
from math import ceil

### initialise search ### 
URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&index=72&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
page = requests.get(URL)

# parse search results card
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id='l-container')

# index for iterator (remember each page has 24 ads)
noOfResults = soup.find('div', attrs={'id': 'searchHeader'})
noOfResults = noOfResults.select('span')[0].text
noOfResults = ceil(int(noOfResults)/24) # round up to nearest page

### Start Loop ###
i = 0
all_ids = np.array([])
adLinks = np.array([])

print(noOfResults, " Pages available")

for x in range(noOfResults):
    URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&index=" + str(i) + "&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
    page = requests.get(URL)

    # parse search results card
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id='l-container')

    # Extract and store ids
    links = results.select('a')
    for card in links:
        ids = card.get('id')
        ids = ids.strip() if ids is not None else ''
        all_ids = np.append(all_ids, ids)

    # clean array and add links
    all_ids = all_ids[all_ids != '']
    cleanIds = np.char.strip(all_ids, 'prop')
    allLinks = np.char.add(cleanIds, '#/?channel=RES_BUY')
    allLinks = np.char.add('https://www.rightmove.co.uk/properties/', allLinks)

    i = i + 24

# removes empty and repeated links 
cleanLinks = np.char.replace(allLinks, 'https://www.rightmove.co.uk/properties/0#/?channel=RES_BUY', '')
cleanLinks = cleanLinks[cleanLinks != '']
cleanLinks = np.unique(cleanLinks)

print(cleanLinks)


10  Pages available
['https://www.rightmove.co.uk/properties/102227564#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/107409182#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/109937678#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/110514809#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/111645674#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/114573920#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/114832574#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/115021484#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/115645691#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116106038#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116106077#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116108690#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116233709#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/116243063#/?channel=RES_BUY'


Fishing Properties

In [77]:
i = 0
for property in cleanLinks:
    ad = requests.get(property)
    soup = BeautifulSoup(ad.content, "html.parser")

    all_info = np.array([])

    # extract property data
    propertyAddress = soup.select('h1')[0].text
    propertyType = soup.select('p')[0].text
    noOfBeds = soup.select('p')[1].text
    noOfBathrooms = soup.select('p')[2].text
    priceLink = soup.body.find('div', attrs={'class': '_1gfnqJ3Vtd1z40MlC0MzXu'})
    price = priceLink.select('span')[0].text
    mapLocation = np.char.add('https://www.rightmove.co.uk/properties/', cleanIds[i])
    floorPlan = np.char.add(mapLocation, '#/floorplan?activePlan=1&channel=RES_BUY')
    mapLocation = np.char.add(mapLocation, '#/map?channel=RES_BUY')
    
    # filter through the images to find the floor plan link
    floorPlanImage = requests.get(floorPlan)
    floorPlanImage = BeautifulSoup(floorPlanImage.content, "html.parser")
    floorPlanImage = floorPlanImage.find('div', attrs={'id': 'root'})
    for j in range(len(floorPlanImage.select('img'))):
        nextFloorPlanImage = floorPlanImage.select('img')[j].get_attribute_list('src',str)
        nextFloorPlanImage = ' '.join(nextFloorPlanImage)
        if '_max_296x197' in nextFloorPlanImage:
            foundFloorPlanImage = nextFloorPlanImage.replace('_max_296x197', '')
            break
        else:
            pass
    
    all_info = np.append(all_info,[price, propertyType, propertyAddress, noOfBeds, noOfBathrooms, foundFloorPlanImage, mapLocation])
    all_info = np.unique(all_info) # use this outside of the for loop at the end.
    i = i+1 
    print(all_info)

['Apartment' 'Echo Court, Telegraph Avenue, NW9'
 'https://media.rightmove.co.uk/dir/88k/87277/126646100/87277_31765566_FLP_00_0000.jpeg'
 'https://www.rightmove.co.uk/properties/126646100#/map?channel=RES_BUY'
 '£475,000' '×1' '×3']
['Flat' 'Watford Way, Hendon, London, NW4'
 'https://media.rightmove.co.uk/dir/12k/11432/126353009/11432_0025_HRT002509770_FLP_00_0000.jpeg'
 'https://www.rightmove.co.uk/properties/126353009#/map?channel=RES_BUY'
 '£320,000' '×1' '×2']
['Flat' 'Sunny Gardens Road, Hendon, NW4'
 'https://media.rightmove.co.uk/dir/124k/123991/85868373/123991_SLD220715_FLP_00_0000.jpeg'
 'https://www.rightmove.co.uk/properties/85868373#/map?channel=RES_BUY'
 '£550,000' '×2' '×4']
['Apartment' 'Hawfinch House, 1 Moorhen Drive, London, NW9'
 'https://media.rightmove.co.uk/dir/250k/249824/125235137/249824_flat_golders_rise_london_greater_london_nw4_2hr_3S_1_FLP_00_0000.jpeg'
 'https://www.rightmove.co.uk/properties/125235137#/map?channel=RES_BUY'
 '£499,000' '×2']
['Flat' 'Gold

KeyboardInterrupt: 

In [6]:
# Load the img

# req = urllib.request.urlopen('https://media.rightmove.co.uk/dir/45k/44870/125628578/44870_31691187_FLP_00_0000.jpeg')
req = urllib.request.urlopen('https://edit.org/photos/img/blog/d3s-design-book-covers.jpg-840.jpg')
arr = np.asarray(bytearray(req.read()))
img = cv2.imdecode(arr, -1) # 'Load it as it is'
imgScaled = cv2.resize(img, (974, 929))  

# Cvt to hsv
hsv = cv2.cvtColor(imgScaled, cv2.COLOR_BGR2HSV)

# Get binary-mask
msk = cv2.inRange(hsv, np.array([0, 0, 175]), np.array([179, 255, 255]))
krn = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 3))
dlt = cv2.dilate(msk, krn, iterations=1)
thr = 255 - cv2.bitwise_and(dlt, msk)

# cv2.imshow('Floor Plan', thr)
# if cv2.waitKey() & 0xff == 27: quit()

# OCR
config = r'--oem 3 --psm 6'
d = pytesseract.image_to_string(thr, config=config)
# print(d)

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.